In [8]:
from models import Complaint
import pickle

# unpickle complaints.pkl
with open('complaints.pkl', 'rb') as f:
    complaints = pickle.load(f)

In [9]:
PAST_MEDICAL_HISTORY_TYPES = {
    "Hypertension",
    "Diabetes mellitus",
    "Hyperlipidemia",
    "Coronary artery disease",
    "Atrial fibrillation",
    "Congestive heart failure",
    "Chronic kidney disease",
    "Chronic obstructive pulmonary disease",
    "Asthma",
    "Stroke",
    "Cancer",
    "Depression",
    "Anxiety",
    "Schizophrenia",
    "Bipolar disorder",
    "Alcohol abuse",
    "Drug abuse",
    "Smoking",
    "Obesity",
    "Edema",
    "Hypothyroidism",
    "Dementia",
    "Pneumonia",
}

In [10]:
NOTE_GEN_SYSTEM_PROMPT = """
You are SynthMedGPT, a bot that generates synthetic medical data. You specifically focus on discarge notes. You insert data that is passed to you, but otherwise you should generate random but sensible data in all other fields. This is the format of your output:

Name:  ___                    Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   ___
 
Service: ___

Allergies: 
___

Attending: ___.
 
Chief Complaint:
___

Major Surgical or Invasive Procedure:
___
 
History of Present Illness:
___
 
Past Medical History:
___
 
Social History:
___

Family History:
___

Physical Exam:
___
 
Pertinent Results:
At Admission:
___
At Discharge:
___
 
Brief Hospital Course:
___
 
Medications on Admission:
___
 
Discharge Medications:
___
 
Discharge Disposition:
___
 
Facility:
___
 
Discharge Diagnosis:
___
 
Discharge Condition:
Mental Status: ___
Level of Consciousness: ___
Activity Status: Ambulatory ___
"""

In [11]:
NOTE_GEN_PROMPT = """
Generate a synthetic discharge note for a patient with the following information:

Chief Complaint: {}
Major Surgical or Invasive Procedure: {}
Past Medical History: {}

All other fields in the discharge note should be randomly generated, but sensible in the overall context.
"""

In [12]:
import random
from openai_client import get_chat_completion

notes = []

def generate_random_note():
    # choose a random complaint from complaints
    complaint = random.choice(complaints)

    # choose a random procedure from complaint.related_procedures
    procedure = random.choice(complaint.related_procedures)

    # choose a random number of medical histories from PAST_MEDICAL_HISTORY_TYPES between 0 and 4
    num_medical_histories = random.randint(0, 4)
    medical_histories = random.sample(PAST_MEDICAL_HISTORY_TYPES, num_medical_histories)

    response = get_chat_completion(
        messages=[
                {"role": "system", "content": NOTE_GEN_SYSTEM_PROMPT},
                {"role": "user", "content": NOTE_GEN_PROMPT.format(complaint, procedure, medical_histories)},
        ]
    )

    return response.choices[0]["message"]["content"]

In [14]:
for i in range(30):
    notes.append(generate_random_note())

/var/folders/l4/7fwz9csx1vx2gyq650rpccn80000gn/T/ipykernel_47879/3981320709.py:15: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  medical_histories = random.sample(PAST_MEDICAL_HISTORY_TYPES, num_medical_histories)


In [16]:
# save the notes in a csv delimited by $

with open('notes.csv', 'w') as f:
    for note in notes:
        f.write(note)
        f.write('$')

In [ ]:
# pickle the notes

with open('notes.pkl', 'wb') as f:
    pickle.dump(notes, f)